# Alteration Filing
## Follow these steps to complete the alteration filing for a corporation in lear

**Note:** Before running these steps this corporation will need to be loaded into lear by using the data_loader located in *lear/lear-db/test_data/data_loader.py*.

## Setup the environment

In [ ]:
%run ./bcr-business-setup.ipynb

## Set the business identifier
**eg. BC1234567**

In [ ]:
identifier = 'BC1234567'

## Verify business info is correct

In [ ]:
business = legal_api.models.Business.find_by_identifier(identifier)
business.id, business.identifier, business.legal_name, business.legal_type

## Fill out the filing json for the alteration

### Base filing json for alteration

In [ ]:
filing_json = {
    'filing': {
        'header': {
            'name': 'alteration',
            'certifiedBy': <name_of_person_filing>,
            'date': <yyyy-mm-dd>,  # today
            'effectiveDate': <datetime>,  # 1970-01-01T00:00:00+00:00
        },
        'business': {
            'identifier': business.identifier,
            'legalName': business.legal_name,
            'legalType': business.legal_type
        },
        'alteration': {
            'provisionsRemoved': <bool>
            'business': {
                'corpType': 'benefitCompany',
            }
        }
    }
}

### Add on name request if required

In [ ]:
name_request = {
    'legalName': <new_name>,
    'nrNumber': <nr_number>,
    'legalType': 'BC'
}
# filing_json['filing']['alteration']['nameRequest'] = name_request

### Add on name translations if required

In [ ]:
# new_name_translations = [<new_translation1>, <etc>]
# modified_translations = [
#     {
#         'oldValue': <old_translation>,
#         'newValue': <new_translation>
#     }
# ]
# ceased_translations = [<existing_translation1>, <etc>]

name_translations = {
    # 'new': new_name_translations,
    # 'modified': modified_translations,
    # 'ceased': ceased_translations
}

# filing_json['filing']['alteration']['nameTranslations'] = name_translations

### Add on alter share structure if required

#### Make base json

In [ ]:
share_structure = {
    'resolutionDates': [<yyyy-mm-dd>, <etc.>],
    'shareClasses': []  # keep this empty
}

#### Create share classes and series json

In [ ]:
new_share_class = {
    'name': <class_name>,
    'priority': <int>,
    'hasMaximumShares': <bool>,
    'maxNumberOfShares': <int or null>,
    'hasParValue': <bool>,
    'parValue': <float>,
    'hasRightsOrRestrictions': <bool>,
    'currency': <string ('CAD')>,
    'series': []  # keep this empty
}

*repeat below for each series in the above class*

In [ ]:
new_share_series = {
    'name': <class_name>,
    'priority': <int>,
    'hasMaximumShares': <bool>,
    'maxNumberOfShares': <int or null>,
    'hasRightsOrRestrictions': <bool>
}
new_share_class['series'].append(new_share_series)

*do not execute until all needed series were added to the new class*

In [ ]:
share_structure['shareClasses'].append(new_share_class)

*only execute after all classes were added (repeat above steps as necessary)* 

In [ ]:
# filing_json['filing']['alteration']['shareStructure'] = share_structure

## Create the alteration filing

In [ ]:
alteration = Filing()
alteration.filing_date = datetime.datetime.utcnow()
alteration.effective_date = datetime.datetime.fromisoformat(filing.filing_json['filing']['header']['effectiveDate'])
alteration.filing_json = filing_json
alteration.business_id = business.id
# alteration.submitter_id = 
# alteration.payment_status_code =
# alteration.payment_token =
# alteration.payment_completion_date =
alteration.save()

## Verify filing info saved correctly


In [ ]:
alteration.filing_date, alteration.effective_date, alteration.business_id, alteration.id, alteration.filing_json

## Enter *./q_cli -f <alteration.id>* into the entity-filer terminal to process the filing

In [ ]:
alteration.id